In [115]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup,NavigableString, Tag
import time
import requests
import re
import csv
from selenium.webdriver.common.by import By
import pandas as pd
import os
import copy


driver = webdriver.Chrome()

driver.get("https://tw.buy.yahoo.com/activity/activity950?p=all2-00-090316-credit-card")
time.sleep(5) #暫停
soup = BeautifulSoup(driver.page_source, 'html.parser')

bank_name= [] #銀行名稱or優惠名稱
result = soup.find('ul', class_='C490B1_items')  
titles = result.find_all('a')

for title in titles:
    #print('☆'+title.text, end="\n")
    #print('☆'+title.get('data-name'), end="\n")
    bank_name.append(title.text)
bank_name.pop(-1)
bank_name.pop(-1)
bank_name.pop(-1)    
print(bank_name)


content_link =[] #存優惠詳情連結
link = soup.find("div", class_='C321B1_wrap')
result = link.find('a')
content_link.append(result.get('href'))
print(content_link[0])
    

#driver = webdriver.Chrome()
driver.get(content_link[0])
time.sleep(5) #暫停
soup = BeautifulSoup(driver.page_source, 'html.parser')


def insert_linka_after_linkc(string1, linkc, linka):
    last_index = -1
    for i in range(len(linkc)):
        index = string1.find(linkc[i], last_index + 1)
        if index != -1:
            last_index = index
            if linkc[i] != linka[i]:  # 判斷 linkc[i] 是否與 linka[i] 相同
                string1 = string1[:index + len(linkc[i])] + '：' + linka[i] + string1[index + len(linkc[i]):]
    return string1


#活動一以外
event_title2 = [] #活動標題
caption1_content2 = [] #活動期間
caption2_content2 = [] #適用通路
caption3_content2 = [] #活動內容
caption4_content2 = [] #登錄辦法
precautions = [] #注意事項

results = soup.find_all('div', class_='C488B1_wrap') #活動框


for result in results:
    result1 = result.find('div', class_='C488B1_title') #活動標題
    #print(result1.text)
    event_title2.append(result1.text)
           
    result4 = result.find('div', class_='C488B1_time_c') #活動期間
    for res in result4:
        #print(res.text)
        caption1_content2.append(res.text) 
    x=1
    result4 = result.find_all('div', class_='C488B1_time_c') #適用通路
    for res in result4:
        if x%2 == 0:
            caption2_content2.append(res.text)  
        x=x+1

    result6 = result.find('div', class_='C488B1_content')
    result5 = result6.find('div', class_='C488B1_content_t') 
    #print(result5.text[:-1])


    result7 = result.find('div', class_='C488B1_notice_r') 
    #print(result7.text[:-1])

    result8 = result.find('div', class_='C488B1_notice_c') #登錄辦法
    #print(result8.text)

    links = result8.find_all('a') #活動中所有連結
    no_link = result8.find('a') #判斷有沒有連結
    linka = [] #一個活動中的所有連結
    linkc = [] #一個活動中在登錄辦法中覆蓋連結的中文
    if no_link != None: #如果有連結
        for link in links:
            #print(link.get('href'))
            linka.append(link.get('href'))
            linkc.append(link.text)
            #print(link.text)
    #print(linka)
    #print(linkc)
    
    new_string = insert_linka_after_linkc(result8.text, linkc, linka)
    caption4_content2.append(new_string)
   
    result5 = result6.find('div', class_='C488B1_content_c') #活動內容
    #print(result5.text)
    #caption3_content2.append(result5.text) 
    
    
#table 

    table = result5.find('table')
    if table:
        div_copy = copy.copy(result5) #活動內容除了table
        div_copy.table.decompose()
        #print(div_copy.get_text(strip=True))
        div_copy = div_copy.get_text(strip=True)
        tr1 = [] #第1列
        tr2 = [] #第2列
        tr3 = [] #第3列
        tr4 = [] #第4列
        tr5 = [] #第5列
        trs = table.find_all('tr')
        n=0
        for tr in trs: #第幾列
            if tr.find('td') != None:
                tds = tr.find_all('td')
                for td in tds: #第幾行
                    if n ==0:
                        tr1.append(td.text.replace("\n", ""))
                    if n ==1:
                        tr2.append(td.text.replace("\n", ""))
                    if n ==2:
                        tr3.append(td.text.replace("\n", ""))
                    if n ==3:
                        tr4.append(td.text.replace("\n", ""))
                    if n ==4:
                        tr5.append(td.text.replace("\n", ""))
                    
                    #print(td.text)
                #print('\n')  
            n+=1
            
        print(tr1)
        print(tr2)
        print(tr3)
        print(tr4)
        print(tr5) 
        
        str1 = tr1[0] + ":"
        str2 = tr2[0] + ':' + tr3[0] + ', '+ tr2[1] + ':' + tr3[1] + ', ' + tr2[2] + ':' + tr3[2]+ ', ' + tr2[3] + ':' + tr3[3]
        str3 = tr2[0] + ':' + tr4[0] + ', '+ tr2[1] + ':' + tr4[1] + ', ' + tr2[2] + ':' + tr4[2]+ ', ' + tr2[3] + ':' + tr4[3]
        str4 = tr2[0] + ':' + tr4[0] + ', '+ tr2[1] + ':' + tr5[0] + ', ' + tr2[2] + ':' + tr5[1]
        
        str_ = [ div_copy, str1, str2, str3, str4 ]
        str_ = '\n'.join(str_)
        #print(str_)
        caption3_content2.append(str_) 
    else: 
        caption3_content2.append(result5.text) 
        
#\table        
        
result11 = soup.find_all('div', class_='C489B1_wrap') #注意事項
k = 1
for res in result11:
    num =  res.find_previous_sibling('a').get('name')
    if  num[0] == '0':
        num = num[1:]
    #print(num)
    #print(str(k)+'k')
    if num != str(k) :
        precautions.append('無')
        #print('無')
        result12 = res.find('div', class_='C489B1_time_c')
        precautions.append(result12.text)
        #print(k)
        k+=1
    else:
        result12 = res.find('div', class_='C489B1_time_c')
        precautions.append(result12.text)
        #print(result12.text)
        #print('有')
    k+=1
    
#print(len(precautions))

bank_dict1 = {
        '銀行名稱':bank_name,
        '注意事項':precautions
}
df = pd.DataFrame(bank_dict1)
#創資料夾
    
folder_name = 'yahoo_購物中心csv'
path = "/Users/takoyaki/Downloads/購物平台爬蟲"  ##要改路徑

folder_path = os.path.join(path, folder_name)
if not os.path.isdir(folder_path):
    os.makedirs(folder_path, mode=0o777)
path0 = str(path+'/'+folder_name+"/")
#print(path0)
df.to_csv(path0+('yahoo_購物中心_注意事項.csv'), index = False)

#for i in range(len(caption4_content2)):
#    print(caption4_content2[i])
#    print('\n\n$')



#print(event_title2)
#print("\n")
#print(len(event_title2))
#print("\n")
#print(caption1_content2)
#print("\n")
#print(len(caption1_content2))
#print("\n")
#print(caption2_content2)
#print("\n")
#print(len(caption2_content2))
#print("\n")
#print(caption3_content2)
#print("\n")
#print(len(caption3_content2))
#print("\n")
#print(caption4_content2)
#print("\n")
#print(len(caption4_content2))


#活動一
event_title = []
event_caption = []

caption1_content = [] #活動期間的內容
caption2_content = [] #適用通路的內容
caption3_content = []
caption4_content = []
#Precautions_content = [] #注意事項的內容 

results = soup.find_all('div', class_='C487B1_wrap') #活動框


for result in results:
    result1 = result.find('div', class_='C487B1_title') #活動標題
    #print(result1.text)
    event_title.append(result1.text)
    result3 = result.find_all('div', class_='C487B1_time_t') #標題:活動期間、適用通路
    for re in result3:
        #print(re.text[:-1])
        if len(event_caption) == 0:
            event_caption.append(re.text[:-1])#append
        elif len(event_caption) == 1:
            event_caption.append(re.text[:-1])
           
    result4 = result.find('div', class_='C487B1_time_c') #內容
    for re in result4:
        #print(re.text)
        caption1_content.append(re.text)#append
    x=1
    result4 = result.find_all('div', class_='C487B1_time_c') 
    for re in result4:
        if x%2 == 0:
            caption2_content.append(re.text)  
        x=x+1

    result6 = result.find('div', class_='C487B1_content')
    result5 = result6.find('div', class_='C487B1_content_t') #標題
    #print(result5.text[:-1])
    if len(event_caption) == 2:
            event_caption.append(result5.text[:-1])#append
    

    result5 = result6.find('div', class_='C487B1_content_c') #內容
    #print(result5.text)
    caption3_content.append(result5.text)#append

    result7 = result.find('div', class_='C487B1_notice_r') #標題
    #print(result7.text[:-1])
    if len(event_caption) == 3:
        event_caption.append(result7.text[:-1])

    result8 = result.find('div', class_='C487B1_notice_c') #內容
    #print(result8.text)
            
    links = result8.find_all('a') #活動中所有連結
    no_link = result8.find('a') #判斷有沒有連結
    linka = [] #一個活動中的所有連結
    linkc = [] #一個活動中在登錄辦法中覆蓋連結的中文
    if no_link != None: #如果有連結
        for link in links:
            #print(link.get('href'))
            linka.append(link.get('href'))
            linkc.append(link.text)
            #print(link.text)
    #print(linka)
    #print(linkc)
    
    new_string = insert_linka_after_linkc(result8.text, linkc, linka)

    caption4_content.append(new_string)
    
    
#print(event_title)
#print("\n")
#print(len(event_title))
#print("\n")
#print(event_caption)
#print("\n")
#print(len(event_caption))
#print("\n")
#print(caption1_content)
#print("\n")
#print(len(caption1_content))
#print("\n")
#print(caption2_content)
#print("\n")
#print(len(caption2_content))
#print("\n")
#print(caption3_content)
#print("\n")
#print(len(caption3_content))
#print("\n")
#print(caption4_content)
#print("\n")
#print(len(caption4_content))


event_num = []
results = soup.find_all('div', class_='C488B1_wrap') #活動框

for result in results:
    num =  result.find_previous_sibling('a').get('name')
    if  num[0] == '0':
        num = num[1:]
    #print(num)
    event_num.append(num)
    
    
events_title = {}
for i, item in enumerate(event_title, 1):
    events_title[f"event_title_{i}"] = [item]  # 創建空的子串列

for j, num in enumerate(event_num):
    key = f"event_title_{num}"
    if key in events_title:
        events_title[key].append(event_title2[j])

#for key, value in events_title.items():
#    print(f"{key}: {value}")
#for value in events_title.values():
#    print(f"Length: {len(value)}")


content1 = {}
for i, item in enumerate(caption1_content, 1):
    content1[f"content1_{i}"] = [item]  # 創建空的子串列

for j, num in enumerate(event_num):
    key = f"content1_{num}"
    if key in content1:
        content1[key].append(caption1_content2[j])

#for key, value in content1.items():
#    print(f"{key}: {value}")
#for value in content1.values():
#    print(f"Length: {len(value)}")


content2 = {}
for i, item in enumerate(caption2_content, 1):
    content2[f"content2_{i}"] = [item]  # 創建空的子串列

for j, num in enumerate(event_num):
    key = f"content2_{num}"
    if key in content2:
        content2[key].append(caption2_content2[j])

#for key, value in content2.items():
#    print(f"{key}: {value}")
#for value in content2.values():
#    print(f"Length: {len(value)}")


content3 = {}
for i, item in enumerate(caption3_content, 1):
    content3[f"content3_{i}"] = [item]  # 創建空的子串列

for j, num in enumerate(event_num):
    key = f"content3_{num}"
    if key in content3:
        content3[key].append(caption3_content2[j])

#for key, value in content3.items():
#    print(f"{key}: {value}")
    
#for value in content3.values():
#    print(f"Length: {len(value)}")


content4 = {}
for i, item in enumerate(caption4_content, 1):
    content4[f"content4_{i}"] = [item]  # 創建空的子串列

for j, num in enumerate(event_num):
    key = f"content4_{num}"
    if key in content4:
        content4[key].append(caption4_content2[j])
        
#for key, value in content4.items():
#    print(f"{key}: {value}")

#for value in content4.values():
#    print(f"Length: {len(value)}")


for i in range(len(events_title)):
    key = f"event_title_{i+1}"
    key1 = f"content1_{i+1}"
    key2 = f"content2_{i+1}"
    key3 = f"content3_{i+1}"
    key4 = f"content4_{i+1}"
    bank_dict = {
        '活動名稱':events_title[key],
        event_caption[0]: content1[key1],
        event_caption[1]: content2[key2],
        event_caption[2]: content3[key3],
        event_caption[3]: content4[key4]
    }
    df = pd.DataFrame(bank_dict)
    
    
    #創資料夾
    
    folder_name = 'yahoo_購物中心csv'
    path = "/Users/takoyaki/Downloads/購物平台爬蟲"  

    folder_path = os.path.join(path, folder_name)
    if not os.path.isdir(folder_path):
        os.makedirs(folder_path, mode=0o777)
    path0 = str(path+'/'+folder_name+"/")
    #print(path0)
    df.to_csv(path0+('yahoo_購物中心'+bank_name[i]+".csv"), index = False)

['YAHOO聯名卡', '永豐', '星展', '華南', '中信', '玉山', '美運', '新光', '彰銀', '聯邦', '遠東', '富邦', '第一', '上海', '安泰', '凱基', '兆豐', '樂天', '合庫', '臺企', '花旗', '國泰', '台新', '匯豐']
https://tw.buy.yahoo.com/activity/activity950?p=act2-z6-b-230606-bankpopindex202307#01
['回饋計算說明']
['回饋單位', '回饋方式', 'Yahoo購物中心', 'Yahoo超級商城', 'Yahoo拍賣']
['中國信託', '超贈點', '2%', '2%', '2%']
['Yahoo', '超贈點', '1%', '1%', '1%']
['購物金(限購衷心會員)', '3%', '-', '-']
